In [1]:
# This notebook is my first attempt to build a NN for classifying the
# Sequences I extracted. I am using Keras because the API is very 
# straightforward. This is an LSTM because recurrent networks were 
# designed to handle variable length inputs, although some people in the 
# keras community appeared to have forgotten that. I think I have an idea
# for the architecture that will handle variable lengths. If this turns
# out to be wrong we can either 1. truncate and pad 2. mask or 3. use a
# different framework

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dropout, Dense
from keras.layers.recurrent import LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional

Using Theano backend.


In [13]:
np.random.seed(42)

In [2]:
malware_classes = ["Agent", "AutoRun", "FraudLoad", "FraudPack", "Hupigon", "Krap",
           "Lipler", "Magania", "None", "Poison", "Swizzor", "Tdss",
           "VB", "Virut", "Zbot"]

# a function for writing predictions in the required format
def write_predictions(predictions, ids, outfile):
    """
    assumes len(predictions) == len(ids), and that predictions[i] is the
    index of the predicted class with the malware_classes list above for
    the executable corresponding to ids[i].
    outfile will be overwritten
    """
    with open(outfile,"w+") as f:
        # write header
        f.write("Id,Prediction\n")
        for i, history_id in enumerate(ids):
            f.write("%s,%d\n" % (history_id, predictions[i]))

def classes_to_Y(classes):
    output = []
    for cls in classes:
        output.append(malware_classes.index(cls))
    return np.array(output)

In [3]:
# load training classes
classes = np.load("../data/features/train_classes.npy")

# load sparse matrix of training data
full_features = np.load("../data/features/100_cutoff_alphabet_19679_word_to_intseq.npy")

In [11]:
# pull out training examples
X = full_features[:classes.shape[0]]

X_test = full_features[classes.shape[0]:]
print X_test.shape

Y = classes_to_Y(classes)

Y_hot = np.zeros((classes.shape[0], 16))
for i, clazz in enumerate(Y):
    Y_hot[i,clazz] = 1

print Y_hot

(3724,)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  1.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [12]:
# Just to check that worked ok.
print classes[21]
print Y[21]
print Y_hot[21]
print len(malware_classes)

None
8
[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
15


In [16]:
# Now randomly select 100 samples to hold out
rand_index = np.random.permutation(np.arange(classes.shape[0]))

X_train = X[rand_index[100:]]
Y_train = Y_hot[rand_index[100:]]
X_validate = X[rand_index[:100]]
Y_validate = Y_hot[rand_index[:100]]


print X_train.shape

print Y_train.shape

print X_validate.shape

print Y_validate.shape
                                   

(2986,)
(2986, 16)
(100,)
(100, 16)


In [32]:
# The vocabulary size is 1 + the maximum integer index. 
vocab_size = 19680

# Length of the dense embedding one for each int in the sequence
embedding_length = 256 # arbitrary

# Must be one to allow variable length
batch_size = 1
model = Sequential()

# Collapse the large input dimension into a 256 dimensional
# dense embedding
model.add(
    Embedding(vocab_size, embedding_length, )
)

# Could add a Dropout layer next but will avoid for now
model.add(Bidirectional(
    LSTM(100, return_sequences=True)
))# Arbitrary output size. TODO make this stateful

# Why not 2!
model.add(Bidirectional(LSTM(42))) # Arbitrary again

model.add(Dense(200, activation="sigmoid"))
model.add(Dense(16, activation="softmax"))

In [33]:
model.compile(loss='categorical_crossentropy',
             optimizer="adam",
             metrics=["accuracy"])

In [22]:
from keras.callbacks import ProgbarLogger, History 

In [29]:
new_X_train = np.array([x.tolist() for x in X_train])

In [34]:
callbacks = [ProgbarLogger(), History()]
model.fit(
    X_train, Y_train, batch_size=1,
    nb_epoch=5, verbose=2, 
    callbacks=callbacks)

Epoch 1/5
Epoch 1/5


ValueError: ('Bad input argument to theano function with name "/n/scrb152/Software/Python/cs181/lib/python2.7/site-packages/keras/backend/theano_backend.py:788"  at index 0(0-based)', 'setting an array element with a sequence.')